In [1]:
import time

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import dump
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)

In [2]:
x_train = np.load('../feature_extraction/ResNext101/features_ResNext101_train.npy')
y_train = np.load('../feature_extraction/ResNext101/labels_ResNext101_train.npy')
x_valid = np.load('../feature_extraction/ResNext101/features_ResNext101_valid.npy')
y_valid = np.load('../feature_extraction/ResNext101/labels_ResNext101_valid.npy')
x_train = np.vstack((x_train, x_valid))
y_train = np.vstack((y_train, y_valid))
n_estimators = 1000
min_samples_leaf = 1
max_features = 'auto'
random_state = 1
rfc = RandomForestClassifier(n_estimators=n_estimators, 
                            min_samples_leaf=min_samples_leaf, 
                            max_features=max_features, 
                            random_state=random_state,
                            n_jobs=64,
                            verbose=0)

rfc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=1000, n_jobs=64, random_state=1)

In [10]:
x_train.shape

(2560, 2048)

In [3]:
x_test = np.load('../feature_extraction/ResNext101/features_ResNext101_test.npy')
y_test = np.load('../feature_extraction/ResNext101/labels_ResNext101_test.npy')
y_test_prob = rfc.predict_proba(x_test)
y_test_prob = np.array(y_test_prob)
predict = y_test_prob[:,:,1].T
predict.shape

(640, 29)

In [4]:
from sklearn.metrics import roc_auc_score, average_precision_score
auc1 = roc_auc_score(y_test[:,0], predict[:,0])
print(f'AUC of Challenge 1: {auc1}')

auc2 = roc_auc_score(y_test[:,1:], predict[:,1:])
print(f'AUC of Challenge 2: {auc2}')

mAP = average_precision_score(y_test[:,1:], predict[:,1:])
print(f'mAP of Challenge 2: {mAP}')

C2_Score = mAP * 0.5 + auc2 * 0.5
print(f'C2 Score: {C2_Score}')
final_Score =  C2_Score * 0.5 + auc1 * 0.5
print(f'C1 Score: {auc1} C2 Score: {C2_Score} Final Score: {final_Score}')

AUC of Challenge 1: 0.9761297268597723
AUC of Challenge 2: 0.9080116113694002
mAP of Challenge 2: 0.5453462397542782
C2 Score: 0.7266789255618392
C1 Score: 0.9761297268597723 C2 Score: 0.7266789255618392 Final Score: 0.8514043262108058


In [5]:
###### Refinement ########
import time

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import dump
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)

from RefinedRandomForest import RefinedRandomForest

In [7]:
c1_scores = []
c2_scores = []
final_scores = []
leaves = []
auc1 = roc_auc_score(y_test[:,0], predict[:,0])
auc2 = roc_auc_score(y_test[:,1:], predict[:,1:])
mAP = average_precision_score(y_test[:,1:], predict[:,1:])
C2_Score = mAP * 0.5 + auc2 * 0.5
final_Score =  C2_Score * 0.5 + auc1 * 0.5
c1_scores.append(auc1)
c2_scores.append(C2_Score)
final_scores.append(final_Score)

In [8]:
####### refine rfc #########
t0 = time.time()
rrfc = RefinedRandomForest(rfc, C = 0.01, n_prunings = 1)
print(f'Time it took for refinement of rfc: {time.time() - t0:.3f} ms.')
leaves.append(sum(rrfc.n_leaves_))
t0 = time.time()
rrfc.fit(x_train, y_train)
# dump(rrfc, f'data/tlrf_rrfc_n_prunings_1.joblib')

# rrfc = joblib.load('data/tlrf_rrfc_n_prunings_1.joblib')
outc = rrfc.predict_proba(x_test)
y_pred_group = outc.argmax(axis=1)

Time it took for refinement of rfc: 2.530 ms.
(2560, 386424)
(2560, 29)
lbfgs


ValueError: y should be a 1d array, got an array of shape (2560, 29) instead.